# Import VM SKU data

In [4]:
import pandas as pd

In [18]:
nsw_co2 = pd.read_csv('./emissions/nsw_co2intensity.csv')
vic_co2 = pd.read_csv('./emissions/vic_co2intensity.csv')

In [66]:
vm_skus_tdp = pd.read_csv('./azurevms/azurevmskus_withtdp.csv')
vm_skus_tdp['skuname'] = vm_skus_tdp['skuname'].str.lower()
vm_skus_tdp

,Unnamed: 0,skutype,skuname,corecount,ram,cpumaxcores,cputdp,memtdp,gpucount,gputdp,cputotaltdp,gputotaltdp,totaltdp
0,0,generalpurpose,standard_dc1s_v2,1,4.0,8,95,1.5000,0.000,0,11.875000,0.0,13.375000
1,1,generalpurpose,standard_dc2s_v2,2,8.0,8,95,3.0000,0.000,0,23.750000,0.0,26.750000
2,2,generalpurpose,standard_dc4s_v2,4,16.0,8,95,6.0000,0.000,0,47.500000,0.0,53.500000
3,3,generalpurpose,standard_dc8_v2,8,32.0,8,95,12.0000,0.000,0,95.000000,0.0,107.000000
4,4,generalpurpose,standard_d1_v2,1,3.5,26,195,1.3125,0.000,0,7.500000,0.0,8.812500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,89,gpuacceleratedcompute,standard_nc24sr_v3,24,448.0,14,135,168.0000,4.000,250,231.428571,1000.0,1399.428571
90,90,gpuacceleratedcompute,standard_nv4as_v4,4,14.0,64,240,5.2500,0.125,300,15.000000,37.5,57.750000
91,91,gpuacceleratedcompute,standard_nv8as_v4,8,28.0,64,240,10.5000,0.250,300,30.000000,75.0,115.500000
92,92,gpuacceleratedcompute,standard_nv16as_v4,16,56.0,64,240,21.0000,0.500,300,60.000000,150.0,231.000000


# 2021 Hackathon - Carbon Footprint Estimator

In [6]:
# These are the inputs

desired_sku_list = [{'sku':'Standard_D3_v2','count':4}, {'sku':'Standard_F16s_v2', 'count':2}, {'sku':'Standard_NC12','count': 1}]
default_hours_month = 730.5
start_month = 10
average_util_pct = .30
desired_region = 'vic'
microsoft_pue_constant = 1.125

In [21]:
if(desired_region == 'nsw'):
    carbonintensity = nsw_co2[['date','CarbonIntensity']]
else:
    carbonintensity = vic_co2[['date','CarbonIntensity']]



In [22]:
# Filter skus
skus_list = [x['sku'] for x in desired_sku_list]
is_in_desired_skus = vm_skus_tdp['skuname'].isin(skus_list)

vm_desired_skus = vm_skus_tdp[is_in_desired_skus]
vm_desired_skus

,Unnamed: 0,skutype,skuname,corecount,ram,cpumaxcores,cputdp,memtdp,gpucount,gputdp,cputotaltdp,gputotaltdp,totaltdp
6,6,generalpurpose,Standard_D3_v2,4,14.0,26,195,5.25,0.0,0,30.0,0.0,35.25
39,39,computeoptimized,Standard_F16s_v2,16,32.0,26,195,12.00,0.0,0,120.0,0.0,132.00
82,82,gpuacceleratedcompute,Standard_NC12,12,112.0,12,135,42.00,2.0,300,135.0,600.0,777.00


In [23]:
# Append SKU count
vm_desired_skus['skucount'] =  vm_desired_skus['skuname'].map(lambda x: [y['count'] for y in desired_sku_list if y['sku'] == x].pop() )
vm_desired_skus['skucount']

/var/folders/kn/y_6jw4b5619752317143js6m0000gn/T/ipykernel_51784/3974041060.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vm_desired_skus['skucount'] =  vm_desired_skus['skuname'].map(lambda x: [y['count'] for y in desired_sku_list if y['sku'] == x].pop() )


6     4
39    2
82    1
Name: skucount, dtype: int64

In [24]:
vm_desired_skus

,Unnamed: 0,skutype,skuname,corecount,ram,cpumaxcores,cputdp,memtdp,gpucount,gputdp,cputotaltdp,gputotaltdp,totaltdp,skucount
6,6,generalpurpose,Standard_D3_v2,4,14.0,26,195,5.25,0.0,0,30.0,0.0,35.25,4
39,39,computeoptimized,Standard_F16s_v2,16,32.0,26,195,12.00,0.0,0,120.0,0.0,132.00,2
82,82,gpuacceleratedcompute,Standard_NC12,12,112.0,12,135,42.00,2.0,300,135.0,600.0,777.00,1


## Calculate total monthly Kwh using formula here (https://devblogs.microsoft.com/sustainable-software/how-can-i-calculate-co2eq-emissions-for-my-azure-vm/)

In [25]:
vm_desired_skus['totalmonthlykwh'] = vm_desired_skus['totaltdp'] * vm_desired_skus['skucount'] * default_hours_month / 1000 * microsoft_pue_constant * average_util_pct
vm_desired_skus['totalmonthlykwh']

/var/folders/kn/y_6jw4b5619752317143js6m0000gn/T/ipykernel_51784/2027254004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vm_desired_skus['totalmonthlykwh'] = vm_desired_skus['totaltdp'] * vm_desired_skus['skucount'] * default_hours_month / 1000 * microsoft_pue_constant * average_util_pct


6      34.762669
39     65.087550
82    191.564494
Name: totalmonthlykwh, dtype: float64

In [26]:
carbonintensity['CarbonIntensity']

0     796.317250
1     818.393298
2     840.086083
3     829.143371
4     856.159838
5     859.364697
6     851.856436
7     844.965079
8     803.094948
9     821.429837
10    786.749184
11    747.712950
12    779.815842
Name: CarbonIntensity, dtype: float64

In [27]:
# Generate CO2eq emissions per month
desired_skus_with_monthly_co2 = vm_desired_skus.apply(lambda x: carbonintensity['CarbonIntensity'].map(lambda y: y * x['totalmonthlykwh']), axis=1)

In [28]:
# Convert to KG
desired_skus_with_monthly_co2 = desired_skus_with_monthly_co2.apply(lambda x: x/1000)

In [29]:
desired_skus_with_monthly_co2

desired_skus_with_monthly_co2['TotalYearlyCO2InKg'] = desired_skus_with_monthly_co2.sum(axis=1)
desired_skus_with_monthly_co2['TotalYearlyCO2InKg']

6      369.704070
39     692.211875
82    2037.305404
Name: TotalYearlyCO2InKg, dtype: float64

In [30]:
desired_skus_with_monthly_co2

,0,1,2,3,4,5,6,7,8,9,10,11,12,TotalYearlyCO2InKg
6,27.682113,28.449535,29.203634,28.823236,29.762401,29.873810,29.612803,29.373241,27.917724,28.555093,27.349501,25.992498,27.108480,369.704070
39,51.830339,53.267215,54.679145,53.966911,55.725346,55.933943,55.445248,54.996707,52.271483,53.464856,51.207577,48.666804,50.756303,692.211875
82,152.546111,156.775098,160.930665,158.834430,164.009826,164.623763,163.185447,161.865308,153.844477,157.356791,150.713209,143.235253,149.385027,2037.305404


In [31]:
vm_desired_skus = vm_desired_skus.join(desired_skus_with_monthly_co2)

In [32]:
vm_desired_skus.head(5)

,Unnamed: 0,skutype,skuname,corecount,ram,cpumaxcores,cputdp,memtdp,gpucount,gputdp,...,4,5,6,7,8,9,10,11,12,TotalYearlyCO2InKg
6,6,generalpurpose,Standard_D3_v2,4,14.0,26,195,5.25,0.0,0,...,29.762401,29.873810,29.612803,29.373241,27.917724,28.555093,27.349501,25.992498,27.108480,369.704070
39,39,computeoptimized,Standard_F16s_v2,16,32.0,26,195,12.00,0.0,0,...,55.725346,55.933943,55.445248,54.996707,52.271483,53.464856,51.207577,48.666804,50.756303,692.211875
82,82,gpuacceleratedcompute,Standard_NC12,12,112.0,12,135,42.00,2.0,300,...,164.009826,164.623763,163.185447,161.865308,153.844477,157.356791,150.713209,143.235253,149.385027,2037.305404


# Microsoft Global Hackathon 2022 - Process test results

## Instructions to run:

* Ensure that the /test_results folder have the test results under the correct subfolder. There are two subfolders (AKS, VM), and under these folders are the regions where the test results have been run.
* Run all cells below here
* Retrieve CSV file output from /eval_output

## Retrieve Carbon Intensity per region

In [100]:
import json

def get_carbon_intensity_for_region(region):
    print('Retrieving intensity for {}'.format(region))
    f = open('./emissions/intensity.json')
    intensities = json.load(f)
    intensity = [i for i in intensities if i['region'] == region][0]
    return intensity['data']['carbonIntensity']


## Define method to retrieve co2eq for 15 minute windows

In [101]:
def get_gco2eqwindow15_for_sku(cpu_util, region, instance_count = 1,skus_list = ['Standard_D3_v2']):
    fn_is_in_desired_skus = vm_skus_tdp['skuname'].isin(skus_list)
    fn_vm_desired_skus = vm_skus_tdp[fn_is_in_desired_skus]
    fn_vm_desired_skus['totalkwh'] = (fn_vm_desired_skus['totaltdp'] * 0.25 / 1000 * microsoft_pue_constant * cpu_util) * instance_count
    intensity = get_carbon_intensity_for_region(region)
    fn_desired_skus_with_monthly_co2 = fn_vm_desired_skus.apply(lambda x: x['totalkwh'] * intensity, axis=1) # change '715' to the actual carbon intensity
    return fn_desired_skus_with_monthly_co2

## Process test_results folder

Process all test results in *test_results* folder. Aggregate results into  *output*

In [103]:
import os

output = []
list_subfolders_with_paths = [f for f in os.scandir('./test_results') if f.is_dir() ]
for workload_type in list_subfolders_with_paths:
    print('Processing {}'.format(workload_type.name))
    subfolders = [f for f in os.scandir(workload_type) if f.is_dir() ]
    for region in subfolders:
        print('Processing {}/{}'.format(workload_type.name,region.name))
        files = [f for f in os.scandir(region) if f.is_file() ]
        for result in files:
            print('Processing {}'.format(result.path))
            tokens = result.name.split('-')
            sku = tokens[2]
            sku = sku.lower()
            print('SKU: {}'.format(sku))
            if(workload_type.name == 'aks'):
                nodes = tokens[3][0]
                print('Nodes: {}'.format(nodes))
                df = pd.read_csv(result.path)
                df.rename(columns={'AggregatedValue': 'AverageCpuUtil'}, inplace=True)
                df = df[['TimeGenerated [UTC]','AverageCpuUtil']]
                df['gCO2Eq'] = df.apply(lambda x: get_gco2eqwindow15_for_sku(x['AverageCpuUtil']/100,region.name, 1,[sku]), axis=1)
                total_gco2eq = df.head(7)['gCO2Eq'].sum()
                print('Total gCO2eq for {} is {}'.format(result.name, total_gco2eq))
                output.append([workload_type.name, region.name, result.name, sku, nodes, total_gco2eq])
            elif(workload_type.name == 'vm'):
                df = pd.read_csv(result.path)
                nodes = 1
                df.rename(columns={'avg_CounterValue': 'AverageCpuUtil'}, inplace=True)
                df = df[df["InstanceName"] == "total"]
                df = df[['TimeGenerated [UTC]','AverageCpuUtil']]
                df['gCO2Eq'] = df.apply(lambda x: get_gco2eqwindow15_for_sku(x['AverageCpuUtil']/100,region.name, 1,[sku]), axis=1)
                total_gco2eq = df.head(7)['gCO2Eq'].sum()
                print('Total gCO2eq for {} is {}'.format(result.name, total_gco2eq))
                output.append([workload_type.name, region.name, result.name, sku, nodes,total_gco2eq])

Processing aks
Processing aks/us_east
Processing aks/us_west
Processing ./test_results/aks/us_west/0920-AKS-Standard_D2_v2-2node-USWest-PK.csv
SKU: standard_d2_v2
Nodes: 2
Retrieving intensity for us_west
Retrieving intensity for us_west
Retrieving intensity for us_west
Retrieving intensity for us_west
Retrieving intensity for us_west
Retrieving intensity for us_west
Retrieving intensity for us_west
Total gCO2eq for 0920-AKS-Standard_D2_v2-2node-USWest-PK.csv is 0.5991953366914812
Processing ./test_results/aks/us_west/0921-AKS-Standard_E2_v4-1node-USWest-PK.csv
SKU: standard_e2_v4
Nodes: 1
Retrieving intensity for us_west
Retrieving intensity for us_west
Total gCO2eq for 0921-AKS-Standard_E2_v4-1node-USWest-PK.csv is 0.20208089420008876
Processing ./test_results/aks/us_west/0919-AKS-Standard_D2_v2-2node-USWest-PK.csv
SKU: standard_d2_v2
Nodes: 2
Retrieving intensity for us_west
Retrieving intensity for us_west
Retrieving intensity for us_west
Retrieving intensity for us_west
Retrieving

In [104]:
output_df = pd.DataFrame(output, columns=['WorkloadType','Region','WorkloadName','Sku','Nodes','TotalgCO2eq'])

In [105]:
output_df['TotalKgCO2eqPerMonth'] = output_df['TotalgCO2eq'] * 96 * 31 / 1000

In [106]:
output_df.to_csv('./eval_output/results.csv')

In [107]:
output_df

,WorkloadType,Region,WorkloadName,Sku,Nodes,TotalgCO2eq,TotalKgCO2eqPerMonth
0,aks,us_west,0920-AKS-Standard_D2_v2-2node-USWest-PK.csv,standard_d2_v2,2,0.599195,1.783205
1,aks,us_west,0921-AKS-Standard_E2_v4-1node-USWest-PK.csv,standard_e2_v4,1,0.202081,0.601393
2,aks,us_west,0919-AKS-Standard_D2_v2-2node-USWest-PK.csv,standard_d2_v2,2,0.507314,1.509766
3,aks,us_west,0921-AKS-Standard_E2_v4-2node-USWest-PK.csv,standard_e2_v4,2,0.673072,2.003063
4,aks,us_west,0921-AKS-Standard_D2_v2-1node-WestUS-PK.csv,standard_d2_v2,1,0.573769,1.707536
5,aks,us_west,0921-AKS-Standard_E2_v4-3Node-USWest-PK.csv,standard_e2_v4,3,0.676079,2.012011
6,aks,us_west,20220921-AKS-Standard_D2_v2-3nodes-WestUS-dere...,standard_d2_v2,3,0.089368,0.265959
7,aks,asia_east,20220920-AKS-Standard_D3_v2-1Node-EastAsia-neh...,standard_d3_v2,1,1.195570,3.558016
8,aks,asia_east,20220920-AKS-Standard_D3_v2-2Node-EastAsia-neh...,standard_d3_v2,2,1.241978,3.696128
9,aks,asia_east,20220920-AKS-Standard_D3_v2-3Node-EastAsia-neh...,standard_d3_v2,3,1.181119,3.515012
